In [ ]:
# !pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet --user # https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/1040332/33

In [17]:
from collections import OrderedDict
import random
import numpy as np
import json
import pickle
import os
import numpy as np
import pandas as pd
import itertools
import openai
api_key = os.getenv("CORRELL_API_KEY")
client = openai.OpenAI(api_key=api_key)
from patient_prompt import prompt as pp
from doctor_prompt import prompt as dp

patient_profiles = pickle.load(open('patient_profiles.pkl', 'rb'))
threshold = 0.8
steps = 5
num_profiles = 1
# resize patient profiles
patient_profiles = OrderedDict(itertools.islice(patient_profiles.items(), num_profiles))

In [18]:
# Function to call OpenAI API
def call_openai(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

# Iterate through patient profiles
for i in patient_profiles:
    profile = patient_profiles[i]
    patient_data = {k: profile[k] for k in list(OrderedDict(profile))[1:-1]}
    doctor_config = {
        "gender": patient_data["gender"],
        "ethnicity": patient_data["ethnicity"],
        "confidence_threshold": threshold,
        "interaction_steps": steps
    }
    
    # Initialize metadata
    metadata = profile['interaction_metadata']
    metadata.update({
        "diagnosis": None,
        "diagnosis_success": False,
        "interaction_duration": 0,
        "num_symptoms_recovered": 0,
        "confidence_history": []
    })
    
    # Format prompts
    pp_copy = pp.format(**patient_data)
    dp_copy = dp.format(**doctor_config)
    
    # Initialize conversation histories
    doctor_history = [{"role": "system", "content": dp_copy}]
    patient_history = [{"role": "system", "content": pp_copy}]
    
    # Run interaction loop
    for step in range(steps):
        metadata["interaction_duration"] += 1
        if metadata["confidence_history"] and metadata["confidence_history"][-1] >= threshold:
            metadata["diagnosis_success"] = True
            break
        
        # Doctor response
        doctor_response = call_openai(doctor_history + [{"role": "user", "content": "" if step == 0 else patient_reply}])
        internal_part, external_part = doctor_response.split("[END INTERNAL]")[-1], doctor_response.split("[BEGIN EXTERNAL]")[-1].strip()
        
        # Update doctor conversation history
        doctor_history.append({"role": "assistant", "content": doctor_response})
        
        # Extract internal reasoning
        internal_data = {}
        try:
            exec(internal_part, {}, internal_data)
            metadata["diagnosis"] = internal_data.get("diagnosis_ranking", [[None]])[0][0]
            metadata["confidence_history"].append(internal_data.get("diagnosis_ranking", [[None, 0]])[0][1])
            metadata["num_symptoms_recovered"] += 1 if any(symptom in patient_reply for symptom in internal_data.get("symptoms_to_verify", [])) else 0
        except:
            pass
        
        # Patient response
        patient_reply = call_openai(patient_history + [{"role": "user", "content": external_part}])
        
        # Update patient conversation history
        patient_history.append({"role": "assistant", "content": patient_reply})
    
    # Store updated metadata
    profile['interaction_metadata'] = metadata

In [19]:
doctor_history

[{'role': 'system',
  'content': '\nYou are a doctor diagnosing a patient by asking questions and gathering information. Follow this structured process:\n\n1. **Internal Thinking (Invisible to the Patient)**\n   - Analyze the patient\'s responses.\n   - Match colloquial symptoms to medical terminology.\n   - Rank possible diagnoses with confidence levels.\n   - Determine what symptom-related questions will help refine the diagnosis.\n   - Maintain a structured thought process without excessive length.\n\n2. **External Interaction (Visible to the Patient)**\n   - Ask the patient symptom-related questions.\n   - Adjust wording to be understandable (use colloquial phrasing).\n   - Acknowledge patient responses appropriately.\n   - Continue until a diagnosis reaches high confidence or N steps are completed.\n\n---\n### **Internal Thought Process (Invisible to Patient) ###\nThe patient will ignore and not consider generations within the [BEGIN INTERNAL] and [END INTERNAL] flags.\n\n[BEGIN I

In [20]:
patient_history

[{'role': 'system',
  'content': '\nYou are a patient interacting with a doctor to discuss your symptoms. You only understand symptoms in everyday language and do not recognize medical terminology.\n\n### **Guidelines for Responding:**\n1. **Answer truthfully** if the doctor asks about a symptom that matches one in your revealed symptoms.\n2. **Deny knowledge** of symptoms you haven’t noticed.\n3. **Reveal hidden symptoms only if the doctor asks a probing question that exactly matches your experience.**  \n   - Example: If you have been experiencing nausea (hidden), and the doctor asks, “Do you feel nauseous?”—you should confirm it.\n4. **Ask for clarification** if the doctor’s question uses unfamiliar medical terms.\n5. **Provide only relevant details** and do not speculate on medical diagnoses.\n6. Ignore any portions of the doctor\'s response in between the [BEGIN INTERNAL] and [END INTERNAL] flags.\n7. Only consider and respond to the doctor\'s responses in between the [BEGIN EXTER

In [21]:
metadata

{'diagnosis': None,
 'diagnosis_success': False,
 'interaction_duration': 5,
 'num_symptoms_recovered': 0,
 'confidence_history': []}